In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url_params = {'txtYEAR':'2019', 'txtMONTH':'01', 'txtGROUPID':'EUCA000500'}
url_headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
url = "https://www.sitca.org.tw/ROC/Industry/IN2422.aspx"

r = requests.get(url, headers = url_headers, params = url_params)
r.encoding = "utf8"
soup = BeautifulSoup(r.text, "lxml")
table = soup.find(id="ctl00_ContentPlaceHolder1_TableClassList")
trs = table.find_all("tr")
newTrs = trs[1].find("table").find_all("tr")
cnt4 = int((len(newTrs)-2)/4)
cnt3 = int((len(newTrs)-2)/3)
cnt2 = int((len(newTrs)-2)/2)

In [20]:
columnName = ['基金名稱','基金英文名','1個月','3個月','6個月','1年','2年','3年','5年','累計','十年','年化標準差','Sharpe']
df = []
for i in range(2, len(newTrs)):
    tds = newTrs[i].find_all("td")
    tmp=[]
    for idx in range(1, len(tds)):
        if(tds[idx].string.strip()==''):
            tmp.append(np.nan)
        elif(idx<3):
            tmp.append(tds[idx].string.strip())  
        else:
            tmp.append(float(tds[idx].string.strip()))  
    df.append(tmp)
df = pd.DataFrame(df, columns = columnName)
'''
df['1個月'] = pd.to_numeric(df['1個月'])
df['3個月'] = pd.to_numeric(df['3個月'])
df['6個月'] = pd.to_numeric(df['6個月'])
df['1年'] = pd.to_numeric(df['1年'])
df['2年'] = pd.to_numeric(df['2年'])
df['3年'] = pd.to_numeric(df['3年'])
df['5年'] = pd.to_numeric(df['1個月'])
df['累計'] = pd.to_numeric(df['累計'])
df['十年'] = pd.to_numeric(df['十年'])
df['年化標準差'] = pd.to_numeric(df['年化標準差'])
df['Sharpe'] = pd.to_numeric(df['Sharpe'])
'''
df.head(10)


,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
0,景順大中華基金A股 美元,Invesco Greater China Equity A USD Acc,5.94,3.62,-10.96,-15.47,21.39,51.05,38.41,5.94,223.19,14.31,0.95
1,瑞銀 (盧森堡) 大中華股票基金 (美元),UBS (Lux) EF Greater China (USD) P,11.02,13.08,-5.19,-14.08,41.03,69.68,75.87,11.02,282.55,16.49,1.08
2,富蘭克林坦伯頓全球投資系列-大中華基金美元A (acc)股,Templeton China A(acc)USD,7.79,7.10,-5.90,-11.76,22.91,53.04,30.65,7.79,120.64,14.65,0.96
3,NN (L) 大中華股票基金P股美元,NN (L) Greater China Equity P Cap USD,7.49,9.34,-3.81,-12.39,19.13,40.61,40.65,7.49,174.66,13.86,0.80
4,富達基金-大中華基金,Fidelity Greater China A-Dis-USD,9.20,9.87,-6.78,-16.56,28.05,59.01,54.30,9.20,247.64,15.95,0.98
5,施羅德環球基金系列－大中華(美元)A1-累積,Schroder ISF Greater China A1 Acc USD,6.98,8.28,-6.75,-14.77,27.04,61.19,53.97,6.98,205.07,15.38,1.04
6,施羅德環球基金系列－大中華(美元)A-累積,Schroder ISF Greater China A Acc USD,7.03,8.42,-6.52,-14.34,28.32,63.63,57.85,7.03,219.49,15.38,1.07
7,施羅德環球基金系列－大中華(美元)C-累積,Schroder ISF Greater China C Acc USD,7.08,8.57,-6.26,-13.87,29.74,66.38,62.47,7.08,240.10,15.39,1.10
8,摩根基金-JPM大中華(美元)-A股(分派),JPM Greater China A (dist) USD,9.49,10.57,-11.88,-23.11,22.53,45.50,36.45,9.49,191.29,17.84,0.72
9,NN (L) 大中華股票基金X股美元,NN (L) Greater China Equity X Cap USD,7.45,9.20,-4.05,-12.83,17.96,38.54,37.21,7.45,161.65,13.85,0.77


## 四四三三法
### 台大教授邱顯比、李存修共同研製
四:1年績效在同類型基金前1/4   
四:2年、3年、5年和今年以來績效在同類型基金前1/4  
三:6個月績效排名在同類型基金前1/3  
三:3個月績效排名在同類型基金前1/3  

In [21]:
# 四四三三法
cnt4 = int((len(newTrs)-2)/4)
cnt3 = int((len(newTrs)-2)/3)
Y1 = df.sort_values(by=['1年'],ascending=False).head(cnt4)
Y2 = df.sort_values(by=['2年'],ascending=False).head(cnt4)
Y3 = df.sort_values(by=['3年'],ascending=False).head(cnt4)
Y5 = df.sort_values(by=['5年'],ascending=False).head(cnt4)
YY = df.sort_values(by=['累計'],ascending=False).head(cnt4)
M3 = df.sort_values(by=['3個月'],ascending=False).head(cnt3)
M6 = df.sort_values(by=['6個月'],ascending=False).head(cnt3)


In [22]:
fund = set(Y1['基金名稱']).intersection(set(Y2['基金名稱'])).intersection(set(Y3['基金名稱'])).intersection(set(YY['基金名稱'])).intersection(set(M3['基金名稱'])).intersection(set(M6['基金名稱']))
fund = list(fund)
fund

['瑞銀 (盧森堡) 大中華股票基金 (美元) I-A1-累積', '瑞銀 (盧森堡) 大中華股票基金 (美元)']

In [23]:
Y1[Y1.基金名稱.isin(fund)]

,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
37,瑞銀 (盧森堡) 大中華股票基金 (美元) I-A1-累積,UBS (Lux) EF Greater China (USD) I-A1,10.87,13.18,-4.64,-13.15,44.50,76.18,87.05,10.87,NaN,16.44,1.16
1,瑞銀 (盧森堡) 大中華股票基金 (美元),UBS (Lux) EF Greater China (USD) P,11.02,13.08,-5.19,-14.08,41.03,69.68,75.87,11.02,282.55,16.49,1.08


## 寬鬆版
### 基金教母蕭碧燕

短期(6個月)、中期(1年)、長期(3年)績效都排在前1/2

In [15]:
M61 = df.sort_values(by=['6個月'],ascending=False).head(cnt2)
Y11 = df.sort_values(by=['1年'],ascending=False).head(cnt2)
Y31 = df.sort_values(by=['3年'],ascending=False).head(cnt2)
Y51 = df.sort_values(by=['5年'],ascending=False).head(cnt2)
fund1 = list(set(M61['基金名稱']).intersection(set(Y11['基金名稱'])).intersection(set(Y31['基金名稱'])))
fund1

['景順大中華基金B股 美元',
 '利安資金中國增長基金(新元)',
 '瀚亞投資-大中華股票基金A(美元)',
 '柏瑞環球基金-柏瑞大中華股票基金 Y',
 '新加坡大華泛華基金',
 'NN (L) 大中華股票基金P股美元',
 '富蘭克林坦伯頓全球投資系列-大中華基金美元B (acc)股',
 'NN (L) 大中華股票基金Y股美元',
 '施羅德環球基金系列－大中華(美元)A-累積',
 '富蘭克林坦伯頓全球投資系列-大中華基金美元A (acc)股',
 '景順大中華基金C股 美元',
 '富達基金-大中華基金(Y類股份累計股份-美元)',
 '元大大中華價值指數基金-人民幣',
 '利安資金中國增長基金(美元)',
 '摩根龍揚基金',
 '瑞銀 (盧森堡) 大中華股票基金 (美元)',
 '富達基金-中國內需消費基金 (歐元)',
 '施羅德環球基金系列－大中華(美元)C-累積',
 '景順大中華基金A股 美元',
 '保德信大中華基金-新台幣',
 '瑞銀 (盧森堡) 大中華股票基金 (美元) I-A1-累積',
 'NN (L) 大中華股票基金X股美元',
 '摩根金龍收成基金',
 '柏瑞環球基金-柏瑞大中華股票基金 A',
 '富達基金-中國內需消費基金 (歐元累積)',
 '施羅德環球基金系列－大中華(美元)A1-累積',
 '富達基金-大中華基金',
 '元大大中華價值指數基金-美元']

In [16]:
FUND21 = Y11[Y11.基金名稱.isin(fund1)]
FUND21

,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
68,元大大中華價值指數基金-人民幣,Yuanta/P-shrs Russell Grt CHN L/C ValCNH,4.12,2.22,-5.77,-9.41,11.40,42.38,NaN,4.12,NaN,11.78,0.97
2,富蘭克林坦伯頓全球投資系列-大中華基金美元A (acc)股,Templeton China A(acc)USD,7.79,7.10,-5.90,-11.76,22.91,53.04,30.65,7.79,120.64,14.65,0.96
3,NN (L) 大中華股票基金P股美元,NN (L) Greater China Equity P Cap USD,7.49,9.34,-3.81,-12.39,19.13,40.61,40.65,7.49,174.66,13.86,0.80
9,NN (L) 大中華股票基金X股美元,NN (L) Greater China Equity X Cap USD,7.45,9.20,-4.05,-12.83,17.96,38.54,37.21,7.45,161.65,13.85,0.77
38,富蘭克林坦伯頓全球投資系列-大中華基金美元B (acc)股,Templeton China B(acc)USD,7.69,6.72,-6.52,-12.89,19.76,47.18,22.37,7.69,NaN,14.62,0.87
37,瑞銀 (盧森堡) 大中華股票基金 (美元) I-A1-累積,UBS (Lux) EF Greater China (USD) I-A1,10.87,13.18,-4.64,-13.15,44.50,76.18,87.05,10.87,NaN,16.44,1.16
46,富達基金-中國內需消費基金 (歐元),Fidelity China Consumer A-Dis-EUR,9.77,9.32,-6.38,-13.26,18.70,42.35,58.91,9.77,NaN,14.44,0.92
47,富達基金-中國內需消費基金 (歐元累積),Fidelity China Consumer A-Acc-EUR,9.79,9.34,-6.35,-13.26,18.75,42.39,59.01,9.79,NaN,14.42,0.92
53,NN (L) 大中華股票基金Y股美元,NN (L) Greater China Equity Y Cap USD,7.36,8.93,-4.54,-13.70,15.61,34.42,30.48,7.36,NaN,13.84,0.70
7,施羅德環球基金系列－大中華(美元)C-累積,Schroder ISF Greater China C Acc USD,7.08,8.57,-6.26,-13.87,29.74,66.38,62.47,7.08,240.10,15.39,1.10


## 年化標準差愈小，波動風險也愈低

In [8]:
FUND21.sort_values(by=['年化標準差'])

,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
68,元大大中華價值指數基金-人民幣,Yuanta/P-shrs Russell Grt CHN L/C ValCNH,4.12,2.22,-5.77,-9.41,11.40,42.38,4.12,4.12,NaN,11.78,0.97
19,保德信大中華基金-新台幣,Prudential Financial Great China TWD,9.33,6.78,-9.32,-18.31,20.54,29.52,9.33,9.33,91.06,13.46,0.63
23,新加坡大華泛華基金,United Greater China,6.44,5.46,-10.40,-17.69,13.62,37.68,6.44,6.44,107.17,13.57,0.80
21,利安資金中國增長基金(新元),LionGlobal China Growth SGD,8.50,6.70,-8.96,-15.02,16.36,43.96,8.50,8.50,115.94,13.81,0.89
53,NN (L) 大中華股票基金Y股美元,NN (L) Greater China Equity Y Cap USD,7.36,8.93,-4.54,-13.70,15.61,34.42,7.36,7.36,NaN,13.84,0.70
9,NN (L) 大中華股票基金X股美元,NN (L) Greater China Equity X Cap USD,7.45,9.20,-4.05,-12.83,17.96,38.54,7.45,7.45,161.65,13.85,0.77
3,NN (L) 大中華股票基金P股美元,NN (L) Greater China Equity P Cap USD,7.49,9.34,-3.81,-12.39,19.13,40.61,7.49,7.49,174.66,13.86,0.80
69,元大大中華價值指數基金-美元,Yuanta/P-shrs Russell Grt CHN L/C ValUSD,6.78,6.25,-4.48,-14.99,13.15,39.90,6.78,6.78,NaN,13.88,0.79
14,景順大中華基金B股 美元,Invesco Greater China Equity B USD Acc,5.87,3.39,-11.36,-16.24,19.18,46.97,5.87,5.87,195.02,14.30,0.89
0,景順大中華基金A股 美元,Invesco Greater China Equity A USD Acc,5.94,3.62,-10.96,-15.47,21.39,51.05,5.94,5.94,223.19,14.31,0.95


## 夏普值(Sharpe Ratio)愈高，可獲得愈高超額報酬

In [9]:
FUND21.sort_values(by=['Sharpe'], ascending=False)

,基金名稱,基金英文名,1個月,3個月,6個月,1年,2年,3年,5年,累計,十年,年化標準差,Sharpe
37,瑞銀 (盧森堡) 大中華股票基金 (美元) I-A1-累積,UBS (Lux) EF Greater China (USD) I-A1,10.87,13.18,-4.64,-13.15,44.50,76.18,10.87,10.87,NaN,16.44,1.16
7,施羅德環球基金系列－大中華(美元)C-累積,Schroder ISF Greater China C Acc USD,7.08,8.57,-6.26,-13.87,29.74,66.38,7.08,7.08,240.10,15.39,1.10
1,瑞銀 (盧森堡) 大中華股票基金 (美元),UBS (Lux) EF Greater China (USD) P,11.02,13.08,-5.19,-14.08,41.03,69.68,11.02,11.02,282.55,16.49,1.08
6,施羅德環球基金系列－大中華(美元)A-累積,Schroder ISF Greater China A Acc USD,7.03,8.42,-6.52,-14.34,28.32,63.63,7.03,7.03,219.49,15.38,1.07
5,施羅德環球基金系列－大中華(美元)A1-累積,Schroder ISF Greater China A1 Acc USD,6.98,8.28,-6.75,-14.77,27.04,61.19,6.98,6.98,205.07,15.38,1.04
26,富達基金-大中華基金(Y類股份累計股份-美元),Fidelity Greater China Y-Acc-USD,9.32,10.13,-6.36,-15.85,30.27,63.00,9.32,9.32,280.43,15.97,1.03
10,景順大中華基金C股 美元,Invesco Greater China Equity C USD Acc,6.00,3.76,-10.72,-14.97,22.83,53.76,6.00,6.00,243.05,14.33,0.99
4,富達基金-大中華基金,Fidelity Greater China A-Dis-USD,9.20,9.87,-6.78,-16.56,28.05,59.01,9.20,9.20,247.64,15.95,0.98
68,元大大中華價值指數基金-人民幣,Yuanta/P-shrs Russell Grt CHN L/C ValCNH,4.12,2.22,-5.77,-9.41,11.40,42.38,4.12,4.12,NaN,11.78,0.97
2,富蘭克林坦伯頓全球投資系列-大中華基金美元A (acc)股,Templeton China A(acc)USD,7.79,7.10,-5.90,-11.76,22.91,53.04,7.79,7.79,120.64,14.65,0.96
